In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, no_op, hog

from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

from menpofit.aam import (
    AAMBuilder,
    LucasKanadeAAMFitter)
from menpofit.lucaskanade import SIC
from menpofit.visualize import visualize_fitting_result, plot_ced

## Load training data

In [ ]:
training_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=None):
    i.rescale_landmarks_to_diagonal_range(200)
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    training_images.append(i)

In [ ]:
visualize_images(training_images)

## Load test data

In [ ]:
test_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/testset/', 
                           verbose=True, max_images=None):    
    i.rescale_landmarks_to_diagonal_range(200)
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    test_images.append(i)

In [ ]:
visualize_images(test_images)

## Build Active Appearance Models

In [ ]:
features = fast_dsift
normalization_diagonal = 100
n_levels = 2
downscale = 2
scaled_shape_models = False
scale_features = True
max_shape_components= 25
max_appearance_components = 200

#### AAM

In [ ]:
aam_builder = AAMBuilder(
    features=features,
    normalization_diagonal=normalization_diagonal, 
    n_levels=n_levels,
    downscale=downscale, 
    scaled_shape_models=scaled_shape_models,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

aam = aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

## Fit Active Appearance Models

In [ ]:
algorithm = SIC

#### AAM Fitter

In [ ]:
aam_fitter = LucasKanadeAAMFitter(
    aam, 
    algorithm=algorithm, 
    n_shape=[5, 15], 
    n_appearance=[50, 100])

#### Fitting loop

In [ ]:
fitters = [aam_fitter]
           
fitters_results = []

for j, i in enumerate(test_images):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    np.random.seed(2)
    s = aam_fitter.noisy_shape_from_shape(gt_s, noise_std=0.02)

    fr = aam_fitter.fit(i, s, gt_shape=gt_s, max_iters=20) 
    fitters_results.append(fr)

    print 'Image: ', j
    print fr

## Results

In [ ]:
initial_errors = [fr.initial_error() for fr in fitters_results]
final_errors = [fr.final_error() for fr in fitters_results]

errors = [initial_errors, final_errors]

In [ ]:
print 'mean_1:', np.mean(final_errors), 'std_1:', np.std(final_errors)

In [ ]:
plot_ced(errors, legend_entries=['ini', 'AAM'])

In [ ]:
visualize_fitting_result(fitters_results)

In [ ]:
%timeit -n 25 fr = aam_fitter.fit(i, s, gt_shape=gt_s, max_iters=20) 